# Analysis

In [1]:
import os
import pandas as pd
from importlib import reload

# Get the current working directory
os.chdir("c:/Users/juanm_8qa8lav/Documents/Proyectos_Personales/FinancialApp")
# Verify the current working directory
print(f"Now in directory: {os.getcwd()}")

# Import your module
from app.modules.database.connection import engine
from app.modules.database.reports import pp, customers, credits, collections, installments, companies, bp, companies, portfolio_inventory
from app.modules.database.credit_manager import credits_balance

balance = credits_balance()
balance['D_Due'] = balance['D_Due'].dt.to_period('D')

print(f'Capital: $ {balance['Capital'].sum():,.2f}')
print(f'Interés: $ {balance['Interest'].sum():,.2f}')
print(f'IVA: $ {balance['IVA'].sum():,.2f}')
print(f'Total: $ {balance['Total'].sum():,.2f}')

Now in directory: c:\Users\juanm_8qa8lav\Documents\Proyectos_Personales\FinancialApp
Capital: $ 34,891,606.64
Interés: $ 31,843,591.08
IVA: $ 6,687,154.04
Total: $ 73,422,351.89


In [2]:
from app.modules.database.reports import pp, customers, credits, collections, installments, companies, bp, companies, portfolio_inventory, fall_inst
from app.modules.database.credit_manager import credits_balance

emission_from = pd.Period("2020/01/01", freq='D')
emission_until = pd.Period.now('D')


fall_inst(emission_from, emission_until, True, True)

,Periodo,Razón Social,Capital,Intereses,IVA,Total
0,2024-10,Asociación Mutual Union Federal,13838.74,43304.82,9094.01,66237.56
1,2024-11,Asociación Mutual Union Federal,37268.87,62242.11,13070.85,112581.82
2,2024-12,Asociación Mutual Union Federal,208289.28,320298.94,67262.79,595851.04
3,2025-01,Asociación Mutual Union Federal,2622107.31,3919332.82,823059.86,7364500.12
4,2025-02,Asociación Mutual Union Federal,2799653.71,4298324.27,902648.07,8000626.07
5,2025-03,Asociación Mutual Union Federal,2923979.74,3893924.69,817724.23,7635628.57
6,2025-04,Asociación Mutual Union Federal,3015300.29,3478512.79,730487.74,7224300.83
7,2025-05,Asociación Mutual Union Federal,2805638.68,3055501.37,641655.33,6502795.28
8,2025-06,Asociación Mutual Union Federal,2632890.65,2670734.94,560854.27,5864479.94
9,2025-07,Asociación Mutual Union Federal,2638066.57,2325991.71,488458.22,5452516.57
